In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from model import VFAE, train
from dataset import DictionaryDataset
from checkpoint import Checkpoint
from data_generator import DataGen
from loss import VFAE_loss
from torch.utils.data import DataLoader

### VAE training

In [2]:
df = pd.read_csv('data/SF_cleaned.csv')
target = 'arrest_made'
sensitive = 'subject_race_black'
features = [col for col in df.columns.tolist() if col not in [target, sensitive]]
x, s, y = df[features], df[sensitive], df[target]

Xtrain, Xtest, strain, stest, ytrain, ytest = train_test_split(x,s,y,test_size=0.3)

In [3]:
train_data = {
    'x': np.array(Xtrain, dtype=np.float32),
    's': np.reshape(np.array(strain, dtype=np.float32), (-1,1),), 
    'y': np.reshape(np.array(ytrain, dtype=np.float32), (-1,1)) 
}

test_data = {
    'x': np.array(Xtest, dtype=np.float32),
    's': np.reshape(np.array(stest, dtype=np.float32), (-1,1),), 
    'y': np.reshape(np.array(ytest, dtype=np.float32), (-1,1)) 
}

In [4]:
# Hyperparams
x_dim = 70
s_dim = 1
y_dim = 1
z1_enc_dim = 100
z2_enc_dim = 100
z1_dec_dim = 100
x_dec_dim = 100
z_dim = 50
dropout_rate = 0.0

batch_size = 100000

alpha = 1
beta = 0.1
gamma = 1
mmd_dims = 500

In [5]:
vfae = VFAE(x_dim, s_dim, y_dim, z1_enc_dim, z2_enc_dim, z1_dec_dim, x_dec_dim, z_dim, dropout_rate, )
optim = torch.optim.Adam(vfae.parameters(),lr=1e-3)

In [6]:
dataset = DictionaryDataset(train_data)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
loss_function = VFAE_loss(alpha=alpha, beta=beta, gamma=gamma, dims_out=mmd_dims)
checkpointer = Checkpoint('checkpoints')

In [7]:
vfae.train()
for e in range(100):
    loss = train(e, vfae, train_dataloader, loss_function, optim, print_freq=250000)
    checkpointer(loss, vfae, e)

Train Epoch: 0 [0/633549 (0%)]	Loss: 26.647087
Train Epoch: 0 Average loss: 22.4119
Train Epoch: 1 [0/633549 (0%)]	Loss: 17.319963
Train Epoch: 1 Average loss: 13.4925
Train Epoch: 2 [0/633549 (0%)]	Loss: 8.125728
Train Epoch: 2 Average loss: 5.0801
Train Epoch: 3 [0/633549 (0%)]	Loss: 3.859965
Train Epoch: 3 Average loss: 10.9840
Train Epoch: 4 [0/633549 (0%)]	Loss: 23.157944


KeyboardInterrupt: 

In [8]:
vfae = VFAE(x_dim, s_dim, y_dim, z1_enc_dim, z2_enc_dim, z1_dec_dim, x_dec_dim, z_dim, dropout_rate, )
generator = DataGen(vfae, 'checkpoints/epoch-2-5.080069541931152.pth',)

In [9]:
test_dataset = DictionaryDataset(test_data)
test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)
output = generator(test_dataloader)

#### Prediction with a classifier head from encoded x

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression

In [12]:
X_train, X_test, y_train, y_test = train_test_split(output['x_pred'],ytest,test_size=0.2)

lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
precision_recall_fscore_support(y_test, y_pred, average='weighted')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.9735498719957536, 0.9866863088113433, 0.9800740737758841, None)